In [60]:
from funciones_helmholtz_bem import Square
from funciones_helmholtz_bem import frequencyToWavenumber
from funciones_helmholtz_bem import solveInteriorBoundary
from funciones_helmholtz_bem import solveInterior
from funciones_helmholtz_bem import printInteriorSolution
import numpy as np
# reload kernel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
# Parámetros físicos del problema
c = 344.0            # Velocidad del sonido en el aire [m/s]
frequency = 400.0    # Frecuencia de la onda [Hz]
density= 1.205        # Densidad del medio [kg/m^3]
k = frequencyToWavenumber(frequency)  # Número de onda correspondiente

# Generación de la malla cuadrada
aVertex, aElement = Square()          # Nodos y elementos de la malla
size = aElement.shape[0]              # Número de elementos en la malla

# Inicialización de los arreglos para las condiciones de contorno
alpha = np.full(size, 1.0, dtype=complex)   # Coeficiente de Robin (α)
beta  = np.full(size, 0.0, dtype=complex)   # Coeficiente de Robin (β)
f     = np.empty(size, dtype=complex)       # Término fuente en la frontera
phi   = np.full(size, 0.0, dtype=complex)   # Potencial escalar (campo total)
v     = np.full(size, 0.0, dtype=complex)   # Derivada normal del potencial

# Cálculo de centros de los elementos y sus longitudes
aCenters = 0.5 * (aVertex[aElement[:, 0]] + aVertex[aElement[:, 1]])
aLength = np.linalg.norm(aVertex[aElement[:, 0]] - aVertex[aElement[:, 1]], axis=1)

# Definición del término fuente f como función del centro de los elementos
f[:] = np.sin(k / np.sqrt(2.0) * aCenters[:, 0]) * np.sin(k / np.sqrt(2.0) * aCenters[:, 1])

# Definición de puntos interiores donde se evaluará la solución
interiorPoints = np.array([
    [0.0250, 0.0250],
    [0.0750, 0.0250],
    [0.0250, 0.0750],
    [0.0750, 0.0750],
    [0.0500, 0.0500]
], dtype=np.float32)

# Potencial incidente en los puntos interiores (inicialmente cero)
interiorIncidentPhi = np.zeros(interiorPoints.shape[0], dtype=complex)

In [62]:
v, phi = solveInteriorBoundary(k, alpha, beta, f, phi, v, c, density, aVertex, aElement)

Density of medium:      1.205 kg/m^3
Speed of sound:         344.0 m/s
Wavenumber (Frequency): 7.306029426953008 (400.0 Hz)

index          Potential                   Pressure                    Velocity              Intensity
    1   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -1.6414e-01+  6.9431e-03i     0.0000e+00
    2   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -5.0024e-01+  8.1575e-03i     0.0000e+00
    3   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -8.3281e-01+  9.1901e-03i     0.0000e+00
    4   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -1.1621e+00+  1.0149e-02i     0.0000e+00
    5   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -1.4863e+00+  1.1014e-02i     0.0000e+00
    6   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -1.8031e+00+  1.1630e-02i     0.0000e+00
    7   0.0000e+00+  0.0000e+00i    0.0000e+00+  0.0000e+00i   -2.1077e+00+  1.1436e-02i     0.0000e+00
    8   0.0000e+00+  0.0000e+00i    0.0000e+

In [63]:
interiorPhi = solveInterior(k, v, phi, interiorIncidentPhi, interiorPoints, aVertex, aElement)

In [64]:
printInteriorSolution(k, c, density, interiorPhi)


Sound pressure at the sample points

index          Potential                    Pressure               Magnitude         Phase

    1   1.5891e-02+  1.1818e-04i   -3.5790e-01+  4.8126e+01i     1.2763e+02 dB       1.5782
    2   4.8175e-02+  4.0339e-05i   -1.2217e-01+  1.4590e+02i     1.3726e+02 dB       1.5716
    3   4.8175e-02+  4.0388e-05i   -1.2232e-01+  1.4590e+02i     1.3726e+02 dB       1.5716
    4   1.4337e-01+ -2.5711e-04i    7.7865e-01+  4.3419e+02i     1.4673e+02 dB       1.5690
    5   6.4983e-02+ -1.3881e-05i    4.2037e-02+  1.9680e+02i     1.3986e+02 dB       1.5706
